In [1]:
# pip install langchain langchain-experimental langchain-openai

In [ ]:
AZURE_OPENAI_API_KEY = "278a70a9e90747dabc92d235cee4772d"
AZURE_OPENAI_ENDPOINT = "https://eastus.api.cognitive.microsoft.com/"

In [7]:
import os 
os.environ['AZURE_OPENAI_API_KEY'] = AZURE_OPENAI_API_KEY
os.environ['AZURE_OPENAI_ENDPOINT'] = AZURE_OPENAI_ENDPOINT


In [8]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(api_version="2024-12-01-preview",model='telcogpt-v1')

prompt = "Tell me a joke about mouse"
model.invoke(prompt)

AIMessage(content='Why did the mouse take a bath?\n\nBecause it wanted to squeak clean! ', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 13, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BZpx4EKq6XsEglk7gliTvbxPanrDe', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'se

In [9]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}"

prompt = ChatPromptTemplate([("system",system_template),("user","{text}")])

prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [10]:
prompt.input_variables

['language', 'text']

In [11]:
prompt.invoke({"language":"Bengali","text":"HOW ARE YOU?"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Bengali', additional_kwargs={}, response_metadata={}), HumanMessage(content='HOW ARE YOU?', additional_kwargs={}, response_metadata={})])

In [12]:
# PARSERS
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
final_prompt = prompt.invoke({"language":"Bengali","text":"HOW ARE YOU?"})
op = model.invoke(final_prompt)
op

AIMessage(content='আপনি কেমন আছেন?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 20, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BZq78vLDSfsfeeGCCNuX5QDAhQ7Kj', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severi

In [14]:
parser.invoke(op)

'আপনি কেমন আছেন?'

In [ ]:
## Chain
- a static sequence of execution involving multiple components such as LLMs, parsers, prompts, tools, loaders etc.
- can be used to automate a rule based, linear and non linear workflows involving LLMs, tools

In [17]:
translation_chain = prompt | model | parser 

In [18]:
translation_chain.invoke({"language":"Bengali","text":"What have you eaten today?"})

'আজ আপনি কি খেয়েছেন?'

## Code Chain

In [19]:
code_generation_prompt = ChatPromptTemplate.from_template("create a python code for {text}. Return only the code and nothing else")

code_chain_01 = code_generation_prompt | model | parser

In [22]:
op = code_chain_01.invoke({"text":"fibonacci series"})
print(op)

```python
def fibonacci(n):
    a, b = 0, 1
    result = []
    for _ in range(n):
        result.append(a)
        a, b = b, a + b
    return result

# Example usage
n = 10  # Change this value for more/less Fibonacci numbers
print(fibonacci(n))
```


In [23]:
code_analyze_prompt = ChatPromptTemplate.from_template("analyze the provided code and calculate the time complexity {code}")
code_chain_02 = code_analyze_prompt | model | parser

In [27]:
def patch_response (code):
    print(code)
    print("#" * 20)
    return {"code":code}
        


In [28]:
final_chain = code_chain_01 | patch_response | code_chain_02

In [29]:
final_chain.invoke({'text': "fibonacci series"})

```python
def fibonacci_series(n):
    fib_series = []
    a, b = 0, 1
    for _ in range(n):
        fib_series.append(a)
        a, b = b, a + b
    return fib_series

# Example usage:
n = 10
print(fibonacci_series(n))
```
####################


"Let's analyze the provided code for the `fibonacci_series` function:\n\n### Code Analysis\n\n1. **Initialization**: \n   ```python\n   fib_series = []\n   a, b = 0, 1\n   ```\n   This initializes an empty list `fib_series` to store Fibonacci numbers, and sets `a = 0` and `b = 1`, which correspond to the first two Fibonacci numbers.\n\n2. **Loop**:\n   ```python\n   for _ in range(n):\n       fib_series.append(a)\n       a, b = b, a + b\n   ```\n   Here, the code uses a for loop that iterates `n` times. In each iteration:\n   - The current value of `a` is appended to `fib_series`.\n   - The values of `a` and `b` are updated such that:\n     - `a` takes the value of `b` (the next Fibonacci number),\n     - `b` takes the value of `a + b` (the sum of the previous two Fibonacci numbers).\n\n### Time Complexity Calculation\n\n- The `for` loop runs `n` times, where `n` is the input to the function.\n- Each iteration of the loop performs a constant amount of work:\n  - Appending a value to a 